In [1]:
### JUPYTER NOTEBOOK / MODULE PATH FIX ###
import sys

if "/share/apps/python/3.8.6/intel/lib/python3.8" in sys.path and sys.path[1] == '':
    sys.path[1], sys.path[6] = sys.path[6], sys.path[1]

In [2]:
from jtb_2022_code.utils.figure_common import *
from jtb_2022_code.utils.adata_common import *
from jtb_2022_code.utils.pseudotime_common import *
from jtb_2022_code import FigureSingleCellData
import numpy as np

In [3]:
## LOAD AND PROCESS UMAP FOR UMAP PANELS ###
data = FigureSingleCellData()

#def remove_velocity(x):
#    del x.layers['denoised_velocity']

#data.apply_inplace_to_expts(calculate_times_velocities, 
#                            layer='denoised',
#                            transform_expr=np.expm1,
#                            distance_key='denoised_distances')

Reading Single Cell Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_1_WT.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_1_fpr1.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_2_WT.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_2_fpr1.h5ad


In [4]:
from inferelator_prior.velocity import decay

In [5]:
data.expt_data[(1, "WT")]

AnnData object with n_obs × n_vars = 63533 × 5747
    obs: 'Gene', 'Replicate', 'Pool', 'Experiment', 'n_counts', 'n_genes', 'CC', 'RP', 'RiBi', 'iESR', 'Mito', 'Cell Cycle', 'Other', 'pca_pt', 'denoised_leiden', 'denoised_n', 'denoised_pca_pt', 'time_pca_pt', 'denoised_cellrank_pt', 'denoised_dpt_pseudotime'
    var: 'n_cells', 'CommonName', 'G1', 'G2', 'M', 'M-G1', 'Mito', 'RP', 'RiBi', 'S', 'iESR', 'Cell Cycle', 'Other', 'denoised_SNR', 'denoised_variance'
    uns: 'CC_colors', 'DESeq_Pseudobulk', 'Pool_colors', 'denoised', 'denoised_SNR', 'denoised_pca', 'denoised_variance', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'denoised_pca'
    varm: 'PCs'
    layers: 'counts', 'denoised', 'denoised_velocity'
    obsp: 'connectivities', 'denoised_connectivities', 'denoised_distances', 'denoised_graph', 'distances'

In [1]:
decays, decays_se, alphas = decay.calc_decay(data.expt_data[(1, "WT")].layers['denoised'], data.expt_data[(1, "WT")].layers['denoised_velocity'])

NameError: name 'decay' is not defined

In [17]:
decays.shape

(5747, 1, 1)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
rgen = np.random.default_rng(100)

panel_genes = ["YPR036W-A", "YOR063W"]

panel_labels = {'expr_11': "A",
                'velocity_11': "B",
                'decay_11': "C"}

panel_titles = {'title_1': "SPO24",
                'title_2': "RPL3"}

x_axis_labelers = {'x_lab_11': ('Time [min]', 0), 
                   'x_lab_21': ('Time [min]', 0.9), 
                   'x_lab_31': ('Expression [counts]', 0.9), 
                   'x_lab_12': ('Time [min]', 0), 
                   'x_lab_22': ('Time [min]', 0.9), 
                   'x_lab_32': ('Expression [counts]', 0.9)}

layout = [['title_1', 'title_1', '.', 'title_2', 'title_2', '.'],
          ['expr_11', 'expr_21', '.', 'expr_12', 'expr_22', 'time_lgd'],
          ['x_lab_11', 'x_lab_11', '.', 'x_lab_12', 'x_lab_12', 'time_lgd'],
          ['.'] * 5 + ['time_lgd'],
          ['velocity_11', 'velocity_21', '.', 'velocity_12', 'velocity_22', 'time_lgd'],
          ['x_lab_21', 'x_lab_21', '.', 'x_lab_22', 'x_lab_22', 'time_lgd'],
          ['decay_11', 'decay_21', '.', 'decay_12', 'decay_22', 'time_lgd'],
          ['x_lab_31', 'x_lab_31', '.', 'x_lab_32', 'x_lab_32', '.'],]

fig_refs = {}

fig, axd = plt.subplot_mosaic(layout,
                              gridspec_kw=dict(width_ratios=[1, 1, 0.25, 1, 1, 0.2], 
                                               height_ratios=[0.05, 1, 0.2, 0.1, 1, 0.2, 1, 0.2],
                                               wspace=0.35), 
                              figsize=(6, 6), dpi=300)

def _minmax(arr):
    return np.min(arr), np.max(arr)

def _minmax_list(lst):
    return min(l[0] for l in lst), max(l[1] for l in lst)

def _get_limits(layer, mod_func=lambda x: x, symmetric=False):
    limits = {g: _minmax_list([_minmax(mod_func(data.expt_data[(j, k)].layers[layer][:, data.expt_data[(j, k)].var_names == g])) 
                              for j in range(1, 3) 
                              for k in ["WT"]]) 
              for g in panel_genes}
    
    if not symmetric:
        return {g: (np.floor(v[0]), np.ceil(v[1])) for g, v in limits.items()}
    else:
        return {g: (-1 * np.max(v), np.max(v)) for g, v in limits.items()}
    

expr_limits = _get_limits('denoised', mod_func = np.expm1)
velocity_limits = _get_limits('denoised_velocity', symmetric=True)

for a in ['title_1', 'title_2', 'time_lgd']:
    axd[a].axis('off')
    
for axid, (lab, yoff) in x_axis_labelers.items():
    axd[axid].axis('off')
    axd[axid].annotate(lab, (0.5, yoff), xycoords='axes fraction', ha='center')


def _minmax(arr):
    return np.min(arr), np.max(arr)

def _minmax_list(lst):
    return min(l[0] for l in lst), max(l[1] for l in lst)

def _get_limits(layer, mod_func=lambda x: x, symmetric=False):
    limits = {g: _minmax_list([_minmax(mod_func(data.expt_data[(j, k)].layers[layer][:, data.expt_data[(j, k)].var_names == g])) 
                              for j in range(1, 3) 
                              for k in ["WT"]]) 
              for g in panel_genes}
    
    if not symmetric:
        return {g: (np.floor(v[0]), np.ceil(v[1])) for g, v in limits.items()}
    else:
        return {g: (-1 * np.max(v), np.max(v)) for g, v in limits.items()}
    

expr_limits = _get_limits('denoised', mod_func = np.expm1)
velocity_limits = _get_limits('denoised_velocity', symmetric=True)

def _replot_axes(ax):
    ax.spines['left'].set_position('zero')
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_position('zero')
    ax.spines['top'].set_color('none')
               
for j in range(1, 3):
    pdata = data.expt_data[(j, "WT")]
    for i, g in enumerate(panel_genes):
        g_idx = pdata.var_names == g
        
        overplot_shuffle = np.arange(pdata.shape[0])
        rgen.shuffle(overplot_shuffle)
        
        time_data = pdata.obs['time_pca_pt'].values[overplot_shuffle]
        expr_data = np.expm1(pdata.layers['denoised'][:, g_idx]).flatten()[overplot_shuffle]
        velocity_data = pdata.layers['denoised_velocity'][:, g_idx].flatten()[overplot_shuffle]
        color_data = pdata.obs['Pool'].map({k: v for k, v in zip(range(1, 9), pool_palette())}).values[overplot_shuffle]
        
        ### PANEL A ###
        a_ref = "expr_" + str(j) + str(i + 1)
        fig_refs[a_ref] = axd[a_ref].scatter(x=time_data, 
                                             y=expr_data,
                                             c=color_data,
                                             alpha=0.25, 
                                             s=2)
        axd[a_ref].set_xlim(0, 80)
        axd[a_ref].set_xticks([0, 80])
        axd[a_ref].set_ylim(*expr_limits[g])
        axd[a_ref].set_yticks([]) if j == 2 else None
        axd[a_ref].set_ylabel("Expression\n[counts]") if j == 1 and i == 0 else None
        axd[a_ref].set_title("Rep." + str(j))
        
        ### PANEL B ###
        b_ref = "velocity_" + str(j) + str(i + 1)
        fig_refs[b_ref] = axd[b_ref].scatter(x=time_data, 
                                             y=velocity_data,
                                             c=color_data,
                                             alpha=0.25, 
                                             s=2)
        
        axd[b_ref].set_xlim(0, 80)
        axd[b_ref].set_xticks([80])
        axd[b_ref].set_ylim(*velocity_limits[g])
        axd[b_ref].set_yticks([]) if j == 2 else None
        axd[b_ref].set_ylabel("Velocity\n[counts/min]") if j == 1 and i == 0 else None

        _replot_axes(axd[b_ref])
        
        ### PANEL C ###
        c_ref = "decay_" + str(j) + str(i + 1)
        fig_refs[c_ref] = axd[c_ref].scatter(x=expr_data, 
                                             y=velocity_data,
                                             c=color_data,
                                             alpha=0.25, 
                                             s=2)
        
        axd[c_ref].set_xlim(*expr_limits[g])
        axd[c_ref].set_xticks([expr_limits[g][1]])
        axd[c_ref].set_ylim(*velocity_limits[g])
        axd[c_ref].set_yticks([]) if j == 2 else None
        axd[c_ref].set_ylabel("Velocity\n[counts/min]") if j == 1 and i == 0 else None

        _replot_axes(axd[c_ref])

        
fig_refs['time_lgd'] = add_legend(axd['time_lgd'], 
                                  pool_palette(), 
                                  data.all_data.obs['Pool'].dtype.categories.values,
                                  title="Time")


for ax_id, label in panel_titles.items():
    axd[ax_id].set_title(label, fontdict={'fontweight': 'bold', 'fontstyle': 'italic'})

for ax_id, label in panel_labels.items():
    axd[ax_id].set_title(label, loc='left', weight='bold')
    
fig.savefig(FIGURE_3_FILE_NAME + ".png", facecolor='white')
        